In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn.preprocessing as pp
import warnings
from scipy.sparse import coo_matrix, csr_matrix, diags
from collections import namedtuple


In [3]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/Users/sahanpramuditha/Downloads/new_id_i.csv')


In [5]:
df.head()

,index,hotelname,tags,starrating,roomtype,country,city,propertytype,user_id
0,0,The Old Cider House,Airconditioning Alarmclock Carpeting Closet Co...,4,Double Room,United Kingdom,Nether Stowey,Hotels,1864
1,1,Apollo Hotel London,Airconditioning Clothesrack Coffee/teamaker Da...,2,Standard Triple Room,United Kingdom,London,Hotels,1693
2,4,Apollo Hotel London,Airconditioning Clothesrack Heating In-roomsaf...,2,Deluxe Triple Room,United Kingdom,London,Hotels,1532
3,7,The Old Cider House,Airconditioning Alarmclock Carpeting Closet Co...,4,Deluxe Double Room with Shower,United Kingdom,Nether Stowey,Hotels,1088
4,8,Hotel Rivoli,Airconditioning Closet Dailyhousekeeping Desk ...,3,Single Room,Germany,Munich,Hotels,1006


In [7]:
df.shape

(43324, 9)

In [15]:
df.duplicated().sum()

0

In [17]:
# Creating an implicit rating system (interaction means a rating of 1)
df['starrating'] = 1

# Create a pivot table (user-hotel matrix)
user_hotel_matrix = df.pivot_table(index='user_id', columns='hotelname', values='starrating', fill_value=0)
user_hotel_matrix.head()


hotelname,1 Elliot Terrace,1 Lexham Gardens Hotel,"100 Queen's Gate Hotel London, Curio Collection by Hilton",14 Reasons Why,1st Floor Koln 2,23 St Mary's,247Hotel,25hours Hotel Bikini Berlin,281 Restaurant and Rooms,3 Paardekens,...,relexa hotel Stuttgarter Hof,rener Hotel Los Campones,room007 Select Sol,room2 Southampton Hometel,t Stadhuys Grote Markt,t Zwanemeer,taverna napoleone,the windsor hotel kirkcaldy,villa natia,waterfront hotel
user_id,,,,,,,,,,,,,,,,,,,,,
1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

class HotelRecSys:
    def __init__(self, user_hotel_matrix):
        self.user_hotel_matrix = user_hotel_matrix
        self.similarity_matrix = None

    def calc_user_user_similarity(self):
        """
        Calculates the cosine similarity between users.
        """
        self.similarity_matrix = cosine_similarity(self.user_hotel_matrix)
        np.fill_diagonal(self.similarity_matrix, 0)  # Ignore similarity with self
        return self.similarity_matrix
    
    def recommend_hotels(self, user_id, k=5):
        """
        Recommend hotels for a given user based on user-user collaborative filtering.
        """
        user_idx = self.user_hotel_matrix.index.get_loc(user_id)
        similar_users = np.argsort(self.similarity_matrix[user_idx])[::-1]
        top_similar_users = similar_users[:k]

        # Find hotels rated by similar users but not by the target user
        user_ratings = self.user_hotel_matrix.iloc[user_idx]
        hotel_scores = np.zeros(self.user_hotel_matrix.shape[1])

        for sim_user_idx in top_similar_users:
            sim_user_ratings = self.user_hotel_matrix.iloc[sim_user_idx]
            hotel_scores += sim_user_ratings

        unrated_hotels = user_ratings[user_ratings == 0].index
        recommendations = [(hotel, score) for hotel, score in zip(self.user_hotel_matrix.columns, hotel_scores) if hotel in unrated_hotels]
        recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)

        return recommendations[:k]

# Initialize recommendation system
rec_sys = HotelRecSys(user_hotel_matrix)
rec_sys.calc_user_user_similarity()


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.04545455],
       [0.        , 0.        , 0.        , ..., 0.023338  , 0.0396059 ,
        0.        ],
       ...,
       [0.        , 0.        , 0.023338  , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.0396059 , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.04545455, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [21]:
# Get top 5 hotel recommendations for a specific user
user_id = 1007  # Replace with the actual user ID
top_recommendations = rec_sys.recommend_hotels(user_id, k=5)

# Print recommended hotels
print(f"Top recommended hotels for User {user_id}:")
for hotel, score in top_recommendations:
    print(f"{hotel} (Score: {score})")


Top recommended hotels for User 1007:
Best Western Hotel Grand Parc Marne La Vallee (ex Chanteloup hotel) (Score: 2.0)
Hotel Panorama (Score: 2.0)
Kyriad Saint Malo Centre - Plage (Score: 2.0)
SUN BEACH (Score: 2.0)
81 The Prom Hotel (Score: 1.0)


In [27]:
import pickle


In [31]:
pickle.dump(df,open('hotellist.pkl', 'wb'))

In [33]:


# Assuming user_hotel_matrix is a DataFrame
with open('user_hotel_matrix.pkl', 'wb') as file:
    pickle.dump(user_hotel_matrix, file)
